In [109]:
import numpy as np
from ortools.linear_solver import pywraplp

In [124]:
#cards + stores + tax information
cards = ["kikijiki", "thoughtseize", "sheoldred", "lili", "kroxa"]
stores = ["TCGPlayer", "401 Games", "Face to Face"]
tax_at_store = [1.13, 1.13, 1.15]

#hold the conversion to cad here
american_or_canadian = [1.3, 1, 1]

#prices vector    

prices_tcg = [18.69, 22.44, 70.05, 13.66, 2.87]
prices_401 = [28, 22, 100, 22, 4.5]
prices_f2f = [34.99, 25 , 110, 22, 5.50]

#shipping fees
shipp_tcg = 8
shipp_401 = 11
shipp_f2f = 5
shipping = [shipp_tcg, shipp_401, shipp_f2f]

cost_matrix = np.array([prices_tcg, prices_401, prices_f2f])

cardsnum = len(cards)
storesnum = len(cost_matrix)

#begin solving process
solver = pywraplp.Solver.CreateSolver("SCIP")

# variable delcaring
# x[i, j] is an array of 0-1 variables
x = {}
for i in range(storesnum):
     for j in range(cardsnum):
          x[i , j] = solver.IntVar(0, 1, cards[j])

#make it so that each card only appears once ----- implementing different conditions would just be making sure the all 
# different condition of the card adds up to one
#for extra copies of the card if they are the same price then can make them add up to the copy of the card 
# otherwise would have to separate them in the cost matrix
for i in range(cardsnum):
    solver.Add(solver.Sum( [x[stores, i] for stores in range(storesnum)]) == 1)

#get the cost function 
cost_fn = []
for i in range(storesnum):
    for j in range(cardsnum):
        cost_fn.append( american_or_canadian[i] * tax_at_store[i] * cost_matrix[i , j] * x[i , j] )

#get the additions to the cost fn of the shipping costs
M = 15000 #should be way more cards than anyone will every order

#is delta 0 or 1 for each store (ie are there cards that need to be shipped by them)
deltas = {}
for i in range(storesnum):
    delname = "delta_" + stores[i]
    deltas[i] = solver.IntVar(0, 1, delname)

for store in range(storesnum):
    solver.Add( 0 <= M * deltas[store] - solver.Sum([x[store,i] for i in range(cardsnum)]))
    solver.Add( M * deltas[store] - solver.Sum([x[store,i] for i in range(cardsnum)]) <= M - 1 )

#add to cost fn
for i in range(storesnum):
    ship_or_not = solver.Sum(x[i, j] for j in range(cardsnum))
    cost_fn.append( american_or_canadian[i] * shipping[i] * deltas[i]) 

#minimize
solver.Minimize(solver.Sum(cost_fn))

status = solver.Solve()

#output
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
     for i in range(storesnum):
         for j in range(cardsnum):
#             # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
             if x[i, j].solution_value() > 0.5:
                print(f"Buy {cards[j]} at {stores[i]}. Card cost is {round(american_or_canadian[i] * tax_at_store[i] * cost_matrix[i][j],2)}.")
   
     for i in range(storesnum):
         print(f"Shipping from {stores[i]} will cost:")
         print(deltas[i].solution_value() * shipping[i])

else:
    print("GUH")

print(f"Total cost = {round(solver.Objective().Value(),2)}\n")

Buy kikijiki at TCGPlayer. Card cost is 27.46.
Buy thoughtseize at TCGPlayer. Card cost is 32.96.
Buy sheoldred at TCGPlayer. Card cost is 102.9.
Buy lili at TCGPlayer. Card cost is 20.07.
Buy kroxa at TCGPlayer. Card cost is 4.22.
Shipping from TCGPlayer will cost:
8.0
Shipping from 401 Games will cost:
0.0
Shipping from Face to Face will cost:
0.0
Total cost = 198.01

